In [1]:
import cv2
import numpy as np
import os

def apply_gaussian_blur(frame, kernel_size=(5, 5)):
    return cv2.GaussianBlur(frame, kernel_size, 0)

def add_gaussian_noise(frame, mean=0, var=5):
    sigma = var ** 0.1
    gauss = np.random.normal(mean, sigma, frame.shape).astype('uint8')
    noisy_frame = cv2.add(frame, gauss)
    return noisy_frame


def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    #                                                                                           
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 使用MP4编码器并保持原视频的帧率
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blurred_frame = apply_gaussian_blur(frame)
        noisy_frame = add_gaussian_noise(blurred_frame)

        out.write(noisy_frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [2]:
from PIL import Image, ImageFilter, ImageEnhance
import numpy as np

def apply_gaussian_blur_gif(frame, radius=5):
    return frame.filter(ImageFilter.GaussianBlur(radius=radius))

def add_gaussian_noise_gif(frame, mean=0, var=5):
    np_frame = np.array(frame)
    sigma = var **  0.1
    gauss = np.random.normal(mean, sigma, np_frame.shape).astype('uint8')
    noisy_frame = cv2.add(np_frame, gauss)
    noisy_frame = np.clip(noisy_frame, 0, 255).astype('uint8')
    return Image.fromarray(noisy_frame)

def process_gif(input_path, output_path):
    gif = Image.open(input_path)
    frames = []
    try:
        while True:
            frame = gif.copy().convert('RGB')
            blurred_frame = apply_gaussian_blur_gif(frame)
            noisy_frame = add_gaussian_noise_gif(blurred_frame)
            frames.append(noisy_frame)  # Convert back to 'P' mode for GIF
            gif.seek(gif.tell() + 1)
    except EOFError:
        pass

    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0)

In [3]:
from tqdm import tqdm
output_dir = '/home/yons/lsy/data/distorted/overall_consistency'
input_dir = '/home/yons/lsy/data/per_dimension'
model_name_list = ['cogvideo','modelscope','videocrafter','lavie']
for model_name in model_name_list:
    input_dir = '/home/yons/lsy/data/per_dimension/'+model_name+'/overall_consistency'
    if model_name == 'cogvideo':
        for gif_name in tqdm(os.listdir(input_dir)):
            input_path = os.path.join(input_dir, gif_name)
            output_path = os.path.join(output_dir, model_name,'overall_consistency', video_name)
            process_gif(input_path, output_path)
    else:
        for video_name in tqdm(os.listdir(input_dir)):
            input_path = os.path.join(input_dir, video_name)
            output_path = os.path.join(output_dir, model_name,'overall_consistency', video_name)
            process_video(input_path, output_path)
    print('finish',model_name)

100%|██████████| 460/460 [13:55<00:00,  1.82s/it]


finish cogvideo


100%|██████████| 460/460 [00:18<00:00, 24.49it/s]


finish modelscope


100%|██████████| 460/460 [00:21<00:00, 21.05it/s]


finish videocrafter


100%|██████████| 460/460 [01:16<00:00,  6.03it/s]

finish lavie


In [1]:
from vbench import VBench
my_VBench = VBench('cuda', "./VBench_full_info.json", "evaluation_results_ditorted")
my_VBench.evaluate(
    videos_path = "/home/yons/lsy/data/distorted/overall_consistency/modelscope/overall_consistency",
    name = "modelscope",
    dimension_list = ["imaging_quality"],
    imaging_quality_preprocessing_mode = "None",
)

/home/yons/anaconda3/envs/lsyllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Evaluation meta data saved to evaluation_results_ditorted/modelscope_full_info.json
cur_full_info_path: evaluation_results_ditorted/modelscope_full_info.json
Loading pretrained model MUSIQ from /home/yons/.cache/vbench/pyiqa_model/musiq_spaq_ckpt-358bb6af.pth


100%|██████████| 460/460 [00:43<00:00, 10.55it/s]

Evaluation results saved to evaluation_results_ditorted/modelscope_eval_results.json


In [3]:
import json
with open('/home/yons/lsy/VideoEvalAgent/GPT4o_results/score/Human_Action_Score_ev_results.json') as f:
    data = json.load(f)

In [33]:
from PIL import Image

def save_frame_from_gif(gif_path, frame_index, output_path):
    gif = Image.open(gif_path)
    try:
        gif.seek(frame_index)
        frame = gif.convert('RGB')
        frame.save(output_path)
        print("Frame saved successfully.")
    except EOFError:
        print("Error: Invalid frame index.")

# 示例用法
gif_path = "/home/yons/lsy/data/per_dimension/cogvideo/human_action/A person is sailing-0.gif"
frame_index = 16
output_path = "/home/yons/lsy/VideoEvalAgent/GPT4o_results/cogvideo_sailing.jpg"

save_frame_from_gif(gif_path, frame_index, output_path)


Frame saved successfully.
